In [ ]:
%%writefile topsis.py
import sys
import pandas as pd
import numpy as np

if len(sys.argv) != 5:
    print("Error: Incorrect number of parameters.")
    sys.exit(1)

input_file = sys.argv[1]
weights = sys.argv[2]
impacts = sys.argv[3]
output_file = sys.argv[4]

try:
    data = pd.read_csv(input_file)
except FileNotFoundError:
    print("Error: File not found.")
    sys.exit(1)

if data.shape[1] < 3:
    print("Error: Input file must contain three or more columns.")
    sys.exit(1)

weights = weights.split(',')
impacts = impacts.split(',')

if len(weights) != len(impacts):
    print("Error: Weights and impacts must be same length.")
    sys.exit(1)

if len(weights) != data.shape[1] - 1:
    print("Error: Number of weights, impacts and columns must be same.")
    sys.exit(1)

for i in impacts:
    if i not in ['+', '-']:
        print("Error: Impacts must be '+' or '-'.")
        sys.exit(1)

for col in data.columns[1:]:
    if not pd.api.types.is_numeric_dtype(data[col]):
        print("Error: All columns except first must be numeric.")
        sys.exit(1)

weights = np.array(weights, dtype=float)
matrix = data.iloc[:, 1:].values

norm = np.sqrt((matrix**2).sum(axis=0))
normalized = matrix / norm

weighted = normalized * weights

ideal_best = []
ideal_worst = []

for i in range(len(impacts)):
    if impacts[i] == '+':
        ideal_best.append(max(weighted[:, i]))
        ideal_worst.append(min(weighted[:, i]))
    else:
        ideal_best.append(min(weighted[:, i]))
        ideal_worst.append(max(weighted[:, i]))

ideal_best = np.array(ideal_best)
ideal_worst = np.array(ideal_worst)

s_plus = np.sqrt(((weighted - ideal_best)**2).sum(axis=1))
s_minus = np.sqrt(((weighted - ideal_worst)**2).sum(axis=1))

score = s_minus / (s_plus + s_minus)

data['Topsis Score'] = score
data['Rank'] = score.argsort()[::-1].argsort() + 1

data.to_csv(output_file, index=False)

print("Result saved successfully!")


Writing topsis.py


In [ ]:
%%writefile data.csv
Fund Name,P1,P2,P3,P4,P5
M1,0.84,0.71,6.7,42.1,12.59
M2,0.91,0.83,7,31.7,10.11
M3,0.79,0.62,4.8,46.7,13.23
M4,0.78,0.61,6.4,42.4,12.55
M5,0.94,0.88,3.6,62.2,16.91
M6,0.88,0.77,6.5,51.5,14.91
M7,0.66,0.44,5.3,48.9,13.83
M8,0.93,0.86,3.4,37,10.55

Overwriting data.csv


In [ ]:
!python topsis.py data.csv "1,1,1,1,1" "+,+,-,+,+" output.csv

Result saved successfully!


In [ ]:
import pandas as pd
pd.read_csv("output.csv")

,Fund Name,P1,P2,P3,P4,P5,Topsis Score,Rank
0,M1,0.84,0.71,6.7,42.1,12.59,0.382109,6
1,M2,0.91,0.83,7.0,31.7,10.11,0.366492,7
2,M3,0.79,0.62,4.8,46.7,13.23,0.496361,4
3,M4,0.78,0.61,6.4,42.4,12.55,0.324792,8
4,M5,0.94,0.88,3.6,62.2,16.91,0.972128,1
5,M6,0.88,0.77,6.5,51.5,14.91,0.547048,3
6,M7,0.66,0.44,5.3,48.9,13.83,0.395015,5
7,M8,0.93,0.86,3.4,37.0,10.55,0.560092,2


In [ ]:
from google.colab import files
files.download("topsis.py")
files.download("output.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>